In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-26 03:06:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-04-26 03:06:35 (11.1 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge-2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RY01SAV7HZ8QO|          4|            0|          0|   N|                Y|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|                N|
|R149Q3B5L33NN5|          5|            0|          0|   N|                Y|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|                Y|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|                Y|
|R3L6FGKAW0EYFI|          1|            1|          1|   N|                N|
| RAO0QZH5VC6VI|          1|            0|          0|   N|                Y|
|R25IK0UAHWNB22|          3|            0|          0|   N|                Y|
|R2A9IHKZMTMAL1|          5|            1|          2|   N|                Y|
| R5XVKTHL6SITI|          5|            0|          0|   N|     

In [8]:
vine_df_filtered = vine_df.filter(vine_df.total_votes >= 20)
vine_df_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
|R1LV778IDGRJQ6|          1|            4|         23|   N|                Y|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R2LBEAGVJA7HK1|          3|            5|         48|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R3QLX5NODD1OJ4|          1|            9|         25|   N|                Y|
|R2SOOHE0XUQON1|          1|            2|         24|   N|     

In [10]:
vine_df_mfifty = vine_df_filtered.filter((vine_df_filtered.helpful_votes / vine_df_filtered.total_votes) >= 0.5)
vine_df_mfifty.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R2I9AO0VEBYJES|          4|           47|         57|   N|                Y|
|R3Q7YJX1I5PDMU|          3|           22|         22|   N|                Y|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R2QJI05ZY7DNSR|          4|           59|         73|   N|     

In [11]:
vine_yes_df = vine_df_mfifty.filter(vine_df_mfifty.vine == "Y")
vine_yes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LLICBKRHLRJC|          4|           17|         22|   Y|                N|
| RUD135S06DEVE|          5|           41|         51|   Y|                N|
|R2WG8UL4SGDPDH|          4|           23|         25|   Y|                N|
|R158AOKOA28WS2|          4|           18|         28|   Y|                N|
| RGWV3QLJQXU9X|          5|          332|        380|   Y|                N|
| RD0E1RZSSYGGN|          5|          261|        292|   Y|                N|
|R3LGCTQH5ZMGO0|          5|           61|         68|   Y|                N|
|R2H24YTRLARCZJ|          4|          277|        297|   Y|                N|
| RYYTC9JTG7WPW|          5|          107|        134|   Y|                N|
| RTB8H4YQYTLY3|          4|           58|         62|   Y|     

In [12]:
vine_no_df = vine_df_mfifty.filter(vine_df_mfifty.vine == "N")
vine_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R2I9AO0VEBYJES|          4|           47|         57|   N|                Y|
|R3Q7YJX1I5PDMU|          3|           22|         22|   N|                Y|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R2QJI05ZY7DNSR|          4|           59|         73|   N|     

In [23]:
star_paid_counts = vine_yes_df.groupby("star_rating").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)","total_paid_star_count")
star_paid_counts.orderBy("star_rating").show()

+-----------+---------------------+
|star_rating|total_paid_star_count|
+-----------+---------------------+
|          1|                    7|
|          2|                    7|
|          3|                   43|
|          4|                   98|
|          5|                  106|
+-----------+---------------------+



In [24]:
star_unpaid_counts = vine_no_df.groupby("star_rating").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)","total_unpaid_star_count")
star_unpaid_counts.orderBy("star_rating").show()

+-----------+-----------------------+
|star_rating|total_unpaid_star_count|
+-----------+-----------------------+
|          1|                   4907|
|          2|                   1625|
|          3|                   2252|
|          4|                   4357|
|          5|                  10899|
+-----------+-----------------------+



In [31]:
star_counts_df = star_paid_counts.join(star_unpaid_counts, ["star_rating"], "inner")
star_counts_df.orderBy("star_rating").show()

+-----------+---------------------+-----------------------+
|star_rating|total_paid_star_count|total_unpaid_star_count|
+-----------+---------------------+-----------------------+
|          1|                    7|                   4907|
|          2|                    7|                   1625|
|          3|                   43|                   2252|
|          4|                   98|                   4357|
|          5|                  106|                  10899|
+-----------+---------------------+-----------------------+



In [35]:
star_counts_df = star_counts_df.withColumn('total_star_count', star_counts_df.total_paid_star_count + star_counts_df.total_unpaid_star_count)
star_counts_df.show()

+-----------+---------------------+-----------------------+----------------+
|star_rating|total_paid_star_count|total_unpaid_star_count|total_star_count|
+-----------+---------------------+-----------------------+----------------+
|          1|                    7|                   4907|            4914|
|          3|                   43|                   2252|            2295|
|          5|                  106|                  10899|           11005|
|          4|                   98|                   4357|            4455|
|          2|                    7|                   1625|            1632|
+-----------+---------------------+-----------------------+----------------+



In [60]:
star_counts_percents = star_counts_df.withColumn("paid_percent", (star_counts_df.total_paid_star_count / 261)*100)
star_counts_percents.show()

+-----------+---------------------+-----------------------+----------------+--------------------+
|star_rating|total_paid_star_count|total_unpaid_star_count|total_star_count|        paid_percent|
+-----------+---------------------+-----------------------+----------------+--------------------+
|          1|                    7|                   4907|            4914|0.028805398954775522|
|          3|                   43|                   2252|            2295|  0.1769474507221925|
|          5|                  106|                  10899|           11005|  0.4361960413151722|
|          4|                   98|                   4357|            4455|  0.4032755853668573|
|          2|                    7|                   1625|            1632|0.028805398954775522|
+-----------+---------------------+-----------------------+----------------+--------------------+



In [65]:
star_counts_percents = star_counts_df.withColumn("unpaid_percent", (star_counts_df.total_unpaid_star_count / 24040)*100).withColumn("paid_percent", (star_counts_df.total_paid_star_count / 261)*100)
star_counts_percents.orderBy("star_rating").show()

+-----------+---------------------+-----------------------+----------------+------------------+------------------+
|star_rating|total_paid_star_count|total_unpaid_star_count|total_star_count|    unpaid_percent|      paid_percent|
+-----------+---------------------+-----------------------+----------------+------------------+------------------+
|          1|                    7|                   4907|            4914|20.411813643926788| 2.681992337164751|
|          2|                    7|                   1625|            1632| 6.759567387687188| 2.681992337164751|
|          3|                   43|                   2252|            2295| 9.367720465890184| 16.47509578544061|
|          4|                   98|                   4357|            4455|18.123960066555743|37.547892720306514|
|          5|                  106|                  10899|           11005|  45.3369384359401| 40.61302681992337|
+-----------+---------------------+-----------------------+----------------+----